In [ ]:
!pip install openmeteo-requests
!pip install requests-cache retry-requests numpy pandas
!pip install holidays

# Описание preprocessing

1. Весь preprocessing сделан в виде последовательного применения функций обработчиков, которые внутри себя копируют исходный массив. Размер наших данных позволял это сделать, но для бОльших объёмов нужно переписывать pipeline обработки так, чтобы он модифицировал датасет in-place.
2. Из внешних источников парсятся погода, выходные, инфляция (отдельным файлом)
3. Документация сделана на английском, но комментарии на русском

- ```split_categories()``` - в колонке "Категория номера" при заказе номеров > 1 через запятую перечисляются все. Чтобы не множить искуственные классы, их имеет смысл разделить.
- ```delete_outliers_iof()``` - применение IsolationForest для фильтрации выбросов в тренировочном датасете. Важно, что в качестве признака включается таргет, поскольку иначе есть опасность отделить в качестве выбросов представителей минорного класса. Параметр *contamination level* подбирался так, чтобы удалялось пропорциональная доля минорного класса.
- ```class_reduction()``` - в колонках "Источник" и "Способ оплаты" встречаются редкие значения, которые были преобразованы в один класс 'minor'.


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
import holidays

In [2]:
train = pd.read_excel('train.xlsx')
test = pd.read_excel('test.xlsx')
rate_inflation = pd.read_excel('rate_inflation.xlsx')

D:\conda\envs\new_env\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
rate_inflation['month'] = rate_inflation['Дата'].apply(lambda x: str(x).split('.')[0]).astype('int32')
rate_inflation['year'] = rate_inflation['Дата'].apply(lambda x: str(x).split('.')[1]).astype('int32')

rate_inflation.loc[len(rate_inflation), ['Ключевая ставка, % годовых', 'Инфляция, % г/г', 'month', 'year']] = [18.0, 9.05, 9, 2024]

In [71]:
rate_inflation

,Дата,"Ключевая ставка, % годовых","Инфляция, % г/г",Цель по инфляции,month,year
0,8.2024,18.0,9.05,4.0,8.0,2024.0
1,7.2024,18.0,9.13,4.0,7.0,2024.0
2,6.2024,16.0,8.59,4.0,6.0,2024.0
3,5.2024,16.0,8.30,4.0,5.0,2024.0
4,4.2024,16.0,7.84,4.0,4.0,2024.0
5,3.2024,16.0,7.72,4.0,3.0,2024.0
6,2.2024,16.0,7.69,4.0,2.0,2024.0
7,1.2024,16.0,7.44,4.0,1.0,2024.0
8,12.2023,16.0,7.42,4.0,12.0,2023.0
9,11.2023,15.0,7.48,4.0,11.0,2023.0


In [3]:
train.head()

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,2,20221204-16563-171020423,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,4,20230529-6634-200121971,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1


In [72]:
def split_categories(train):
    '''
    Splitting columns "Категория номера" to one-hot vector 
    '''
    df = train.copy()
    categories = df[df['Номеров'] == 1]['Категория номера'].unique()

    for i, category in enumerate(categories):
        df[f'Категория номера_{category}'] = df['Категория номера'].apply(lambda x: 1 if category in x else 0)

    
    return df.drop('Категория номера', axis = 1)
        

In [ ]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry


def parse_weather(lat, long):
    '''
    Returns dataset with weather from 2020-10-10 to 2024-09-20. Code was taken from https://open-meteo.com/
    '''
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
    	"latitude": lat,
    	"longitude": long,
    	"start_date": "2020-10-10",
    	"end_date": "2024-09-20",
    	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "wind_speed_10m"],
    	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "precipitation_sum"],
    	"timezone": "Europe/Moscow"
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")
    
    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
    daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
    daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
    daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(7).ValuesAsNumpy()
    
    daily_data = {"date": pd.date_range(
    	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    	freq = pd.Timedelta(seconds = daily.Interval()),
    	inclusive = "left"
    )}
    daily_data["weather_code"] = daily_weather_code
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
    daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
    daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
    daily_data["precipitation_sum"] = daily_precipitation_sum
    
    daily_dataframe = pd.DataFrame(data = daily_data)
    return daily_dataframe

def add_temp(train):
    '''
    Add colunmns with weather characteritics for the nearest date from parsed sets. 
    '''
    df = train.copy()
    # NB! Weather code - категориальный признак!
    coords = {1: (60.515121, 30.214933), 3: (61.820041, 30.633036), 2: (61.041992, 30.178778), 4: (59.232684, 29.197951)}

    # Предспаршенные датасеты с колонкой 
    first_set = parse_weather(coords[1][0], coords[1][1])
    second_set = parse_weather(coords[2][0], coords[2][1])
    third_set = parse_weather(coords[3][0], coords[3][1])
    fourth_set = parse_weather(coords[4][0], coords[4][1])
    sets = [first_set, second_set, third_set, fourth_set]

    for set in sets:
        set['date'] = set['date'].apply(lambda x: pd.Timestamp(pd.to_datetime(x)).tz_localize(None))

    def find_nearest_date(date, hostel_number):
        date_set = sets[hostel_number-1]
        weather = date_set.iloc[np.abs(date_set.date - date).idxmin()].values[1:]
        
        
        return pd.Series(weather, index=[
            'weather_code', 'temperature_2m_max', 'temperature_2m_min', 
            'temperature_2m_mean', 'apparent_temperature_max', 
            'apparent_temperature_min', 'apparent_temperature_mean', 
            'precipitation_sum'])

    # Колонки для погоды
    time_cols = ['Дата бронирования', 'Заезд', 'Выезд']
    weather_cols = ['_weather_code', '_temperature_2m_max', '_temperature_2m_min', 
                    '_temperature_2m_mean', '_apparent_temperature_max', 
                    '_apparent_temperature_min', '_apparent_temperature_mean', 
                    '_precipitation_sum']
    
    # Применяем функцию для каждой строки и создаем новые столбцы
    for col in time_cols:
        weather_data = df.apply(lambda row: find_nearest_date(row[col], int(row['Гостиница'])), axis=1)
        
        # Добавляем новые столбцы в DataFrame
        for weather_col, weather_data_col in zip(weather_cols, weather_data.columns):
            df[f'{col}{weather_col}'] = weather_data[weather_data_col]

    return df

In [5]:
def delete_outliers_iof(train):
    '''
    Do sklearn.ensemble.IsolationForest for deleting outliers
    '''
    df_train = train.copy()
    df_train = df_train.drop(["№ брони", "Дата отмены", "Статус брони", "Unnamed: 0"], axis = 1)
    cat_features = df_train.select_dtypes('object')
    
    encoder = LabelEncoder()
    for col in cat_features:
        df_train[col] = encoder.fit_transform(df_train[col])
    
    cols_to_drop = df_train.select_dtypes("datetime64").columns
    df_train = df_train.drop(cols_to_drop, axis = 1)

    iof = IsolationForest(contamination = 0.02) # contamination level был подобран эвристично
    iof.fit(df_train)
    predictions = iof.predict(df_train)

    return train.drop(np.where(predictions < 0)[0], axis = 0)

# delta1_time_hours - между датой бронирования и заездом
# delta2_time_hours - между выездом и заездом
def process_time(train, is_test = False):
    '''
    Generate new features:
    - day, week, day of the week, year ...
    - delta between times in hours
    '''
        
    df_train = train.copy()
    if not is_test:
        df_train.drop('Дата отмены', axis = 1)
    time_columns = df_train.select_dtypes("datetime64").columns

    for col in time_columns:
        df_train[f"{col}_month"]= df_train[col].dt.month
        df_train[f"{col}_day"] = df_train[col].dt.day
        df_train[f"{col}_year"] = df_train[col].dt.year
        df_train[f"{col}_is_holiday"] = df_train[col].apply(lambda x: 1 if x in holidays.RU() else 0)
        df_train[f"{col}_weekday"] = df_train[col].dt.weekday

    
    df_train["delta1_time_hours"] = (df_train["Заезд"] - df_train["Дата бронирования"]).apply(lambda x: x.seconds / 60)
    df_train["delta2_time_hours"] = (df_train["Выезд"] - df_train["Заезд"]).apply(lambda x: x.seconds / 60)

    
    
    return df_train#.drop(time_columns, axis = 1)


def generate_features(train):
    '''
    Additional numerical features
    '''
    df_train = train.copy()

    df_train["original_index"] = df_train.index


    df_train["Предоплата/Стоимость"] = df_train["Внесена предоплата"] / df_train["Стоимость"]
    df_train["Стоимость/Гости"] = df_train["Стоимость"] / df_train["Гостей"]
    df_train["Стоимость/Ночь"] = df_train["Стоимость"] / df_train["Ночей"]

    average_prices = df_train.groupby('Категория номера')['Стоимость'].mean().reset_index()
    average_prices.columns = ['Категория номера', 'Средняя стоимость']

    average_prices_by_month = df_train.groupby('Заезд_month')['Стоимость'].mean().reset_index()
    average_prices_by_month.columns = ['Заезд_month', 'Средняя стоимость_month']
    
    # 2. Объединим средние стоимости с исходнми данными
    df_train = df_train.merge(average_prices, on='Категория номера')
    df_train = df_train.merge(average_prices_by_month, on = "Заезд_month")
    
    # 3. Рассчитаем отклонение от средней стоимости
    df_train['Отклонение_по_номеру'] = df_train['Стоимость'] - df_train['Средняя стоимость']
    df_train["Отклонение_по_месяцу"] = df_train['Стоимость'] - df_train['Средняя стоимость_month']

    df_train = df_train.sort_values(by="original_index").drop("original_index", axis=1)

    for i, row in df_train.iterrows():
        try:
            rate_book = rate_inflation[(rate_inflation['month'] == row['Дата бронирования_month']) & (rate_inflation['year'] == row['Дата бронирования_year'])]['Ключевая ставка, % годовых'].values[0]
            rate_enter = rate_inflation[(rate_inflation['month'] == row['Заезд_month']) & (rate_inflation['year'] == row['Заезд_year'])]['Ключевая ставка, % годовых'].values[0]
            inflation_enter = rate_inflation[(rate_inflation['month'] == row['Заезд_month']) & (rate_inflation['year'] == row['Заезд_year'])]['Инфляция, % г/г'].values[0]
            inflation_book = rate_inflation[(rate_inflation['month'] == row['Дата бронирования_month']) & (rate_inflation['year'] == row['Дата бронирования_year'])]['Инфляция, % г/г'].values[0]
        except:
            print(row['Заезд_month'], row['Заезд_year'])

        df_train.loc[i, 'rate_book'] = rate_book
        df_train.loc[i, 'rate_enter'] = rate_enter
        df_train.loc[i, 'inflation_enter'] = inflation_enter
        df_train.loc[i, 'inflation_book'] = inflation_book

    to_drop = ['Средняя стоимость',"Средняя стоимость_month"]
    return df_train.drop(to_drop , axis = 1)

def class_reduction(train):
    '''
    Merge minor categories in columns "Способ оплаты", "Источник"
    '''
    df = train.copy()
    
    to_eliminate = ['Zabroniryi.ru',
     'Cuva',
     'Acase.ru (ООО "АКАДЕМ-ОНЛАЙН")',
     'Acase.ru (ООО "ПРАНДИУМ")',
     'Alean.ru (13.10.2023-02.06.2025)',
     'Alean.ru (03.02.2022-31.01.2023)',
     '101hotels.com',
     'Acase.ru (ООО "КАЛЕЙДОСКОП")',
     'Alean.ru (16.01.2023-06.09.2024)',
     'Bronevik.com/Bro.Online',
     'Тинькофф Путешествия',
     'ВКонтакте',
     'Otello',
     'expedia.com (A-Hotels.com)',
     'Ozon',
     'Alean.ru (31.10.2023-22.06.2025)',
     'Svoy Hotel',
     'Alean.ru (20.01.2021-31.01.2022)']
    
    to_eliminate_payment = ['Банк. карта: Эквайринг ComfortBooking (Банк. карта)',
     'Банк. карта (SberPay): Эквайринг ComfortBooking (Банк. карта) (SberPay)',
     'Внешняя система оплаты (Банковская карта)',
     'Банк. карта (Yandex Pay): Эквайринг ComfortBooking (Банк. карта) (Yandex Pay)']
    
    df['Источник'] = df['Источник'].apply(lambda x: 'minor_class' if x in to_eliminate else x)
    df['Способ оплаты'] = df['Способ оплаты'].apply(lambda x: 'minor_class' if x in to_eliminate_payment else x)
    
    return df


In [6]:
def preprocess_set(train, is_test = False):
    df = train.copy()
    if not is_test:
        df = delete_outliers_iof(df)
        df['Дата отмены'].fillna(0, inplace = True)
        df['target'] = df['Дата отмены'].apply(lambda x: 1 if x != 0 else 0)
    df = add_temp(df)
    df = process_time(df, is_test)
    df = generate_features(df)
    #df = split_categories(df)
    df = class_reduction(df)

    if not is_test:
        return df.drop(['Unnamed: 0', '№ брони', 'Дата отмены', 'Статус брони'], axis = 1)
    else:
        return df.drop(['Unnamed: 0', '№ брони'], axis = 1)

In [10]:
df = preprocess_set(train)
df.to_csv("Train.csv", index = False)

Coordinates 60.492088317871094°N 30.141510009765625°E
Elevation 167.0 m asl
Timezone b'Europe/Moscow' b'MSK'
Timezone difference to GMT+0 10800 s
Coordinates 61.054481506347656°N 30.07211685180664°E
Elevation 19.0 m asl
Timezone b'Europe/Moscow' b'MSK'
Timezone difference to GMT+0 10800 s
Coordinates 61.82776641845703°N 30.66666603088379°E
Elevation 80.0 m asl
Timezone b'Europe/Moscow' b'MSK'
Timezone difference to GMT+0 10800 s
Coordinates 59.22671127319336°N 29.117647171020508°E
Elevation 68.0 m asl
Timezone b'Europe/Moscow' b'MSK'
Timezone difference to GMT+0 10800 s


In [11]:
df = preprocess_set(test, is_test = True)
df.to_csv("Test.csv", index = False)

Coordinates 60.492088317871094°N 30.141510009765625°E
Elevation 167.0 m asl
Timezone b'Europe/Moscow' b'MSK'
Timezone difference to GMT+0 10800 s
Coordinates 61.054481506347656°N 30.07211685180664°E
Elevation 19.0 m asl
Timezone b'Europe/Moscow' b'MSK'
Timezone difference to GMT+0 10800 s
Coordinates 61.82776641845703°N 30.66666603088379°E
Elevation 80.0 m asl
Timezone b'Europe/Moscow' b'MSK'
Timezone difference to GMT+0 10800 s
Coordinates 59.22671127319336°N 29.117647171020508°E
Elevation 68.0 m asl
Timezone b'Europe/Moscow' b'MSK'
Timezone difference to GMT+0 10800 s


In [75]:
df.head(5)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,...,rate_enter,inflation_enter,inflation_book,Категория номера_Номер «Студия»,Категория номера_Номер «Стандарт»,Категория номера_Апартаменты с 2 спальнями с отдельным входом,Категория номера_Коттедж с 3 спальнями,Категория номера_Коттедж с 2 спальнями,Категория номера_Номер «Люкс»,Категория номера_Номер «Стандарт» для маломобильных групп населения
0,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,...,15.0,7.48,7.48,1,0,0,0,0,0,0
10641,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,...,7.5,11.94,11.94,0,1,0,0,0,0,0
10642,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,...,7.5,11.94,11.98,0,1,0,0,0,0,0
3878,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,...,12.0,5.15,4.30,0,0,1,0,0,0,0
8081,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,...,7.5,3.51,3.51,0,1,0,0,0,0,0


In [23]:
test.head(5)

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,0,20231129-16563-238946689,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,20221219-7491-174959103,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,2,20221211-6634-172724329,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,3,20230821-6634-212247350,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,4,20230326-6634-189784563,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


In [65]:
test['Заезд']

0

## Погода

In [8]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry


def parse_weather(lat, long):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)
    
    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
    	"latitude": lat,
    	"longitude": long,
    	"start_date": "2020-10-10",
    	"end_date": "2024-09-20",
    	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "wind_speed_10m"],
    	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "precipitation_sum"],
    	"timezone": "Europe/Moscow"
    }
    responses = openmeteo.weather_api(url, params=params)
    
    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")
    
    # Process daily data. The order of variables needs to be the same as requested.
    daily = response.Daily()
    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
    daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
    daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
    daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(7).ValuesAsNumpy()
    
    daily_data = {"date": pd.date_range(
    	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
    	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
    	freq = pd.Timedelta(seconds = daily.Interval()),
    	inclusive = "left"
    )}
    daily_data["weather_code"] = daily_weather_code
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
    daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
    daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
    daily_data["precipitation_sum"] = daily_precipitation_sum
    
    daily_dataframe = pd.DataFrame(data = daily_data)
    return daily_dataframe

In [9]:
def add_temp(train):
    df = train.copy()
    # NB! Weather code - категориальный признак!
    coords = {1: (60.515121, 30.214933), 3: (61.820041, 30.633036), 2: (61.041992, 30.178778), 4: (59.232684, 29.197951)}

    first_set = parse_weather(coords[1][0], coords[1][1])
    second_set = parse_weather(coords[2][0], coords[2][1])
    third_set = parse_weather(coords[3][0], coords[3][1])
    fourth_set = parse_weather(coords[4][0], coords[4][1])
    sets = [first_set, second_set, third_set, fourth_set]

    for set in sets:
        set['date'] = set['date'].apply(lambda x: pd.Timestamp(pd.to_datetime(x)).tz_localize(None))

    def find_nearest_date(date, hostel_number):
        date_set = sets[hostel_number-1]
        weather = date_set.iloc[np.abs(date_set.date - date).idxmin()].values[1:]
        
        # Убедись, что результат имеет правильную длину
        return pd.Series(weather, index=[
            'weather_code', 'temperature_2m_max', 'temperature_2m_min', 
            'temperature_2m_mean', 'apparent_temperature_max', 
            'apparent_temperature_min', 'apparent_temperature_mean', 
            'precipitation_sum'])

    # Колонки для погоды
    time_cols = ['Дата бронирования', 'Заезд', 'Выезд']
    weather_cols = ['_weather_code', '_temperature_2m_max', '_temperature_2m_min', 
                    '_temperature_2m_mean', '_apparent_temperature_max', 
                    '_apparent_temperature_min', '_apparent_temperature_mean', 
                    '_precipitation_sum']
    
    # Применяем функцию для каждой строки и создаем новые столбцы
    for col in time_cols:
        weather_data = df.apply(lambda row: find_nearest_date(row[col], int(row['Гостиница'])), axis=1)
        
        # Добавляем новые столбцы в DataFrame
        for weather_col, weather_data_col in zip(weather_cols, weather_data.columns):
            df[f'{col}{weather_col}'] = weather_data[weather_data_col]

    return df

In [54]:
date = train['Дата бронирования'][0]#.dt.strftime('%Y-%d-%m')
daily_dataframe.iloc[np.abs(daily_dataframe.date- date).idxmin()]

date                         2023-04-20 21:00:00
weather_code                                 3.0
temperature_2m_max                        13.662
temperature_2m_min                         2.462
temperature_2m_mean                     8.461999
apparent_temperature_max               10.799542
apparent_temperature_min               -0.908784
apparent_temperature_mean               5.640798
precipitation_sum                            0.0
Name: 923, dtype: object

In [48]:
daily_dataframe['date'] = daily_dataframe['date'].apply(lambda x: pd.Timestamp(pd.to_datetime(x)).tz_localize(None))

In [42]:
pd.Timestamp(pd.to_datetime(daily_dataframe.date[0])).tz_localize(None)

Timestamp('2020-10-09 21:00:00')